In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as im
import os
import pandas as pd
import cv2
import math
from skimage.color import rgb2gray
from ImageGenerator import ImageDataGenerator, ImageAugmentation
from Preprocessing import EdgeDetection, Enhancement, Morphology
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

ModuleNotFoundError: No module named 'Preprocessing'

## Data Loading

In [ ]:
def display_sample(images, labels):
    plt.figure(figsize=(20, 10))
    for i in range(7):
        plt.subplot(1, 7, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i], cmap='gray')
        plt.xlabel(f'ID`  {labels[i]}')
    plt.show()

In [ ]:
datagen = ImageDataGenerator()
images, labels = datagen.flow_from_directory(
    directory = 'Finger Dataset',
    target_size=(300, 300),
    shuffle=True
)

In [ ]:
images.shape, labels.shape

In [ ]:
count_classes = np.unique(labels, return_counts=True)
plt.bar(count_classes[0], count_classes[1])

In [ ]:
display_sample(images=images, labels=labels)

## Data Preparation

### Downsampling

In [2]:
def downsample(images, labels, n_samples):
    unique_labels = np.unique(labels)
    downsampled_images = []
    downsampled_labels = []
    for label in unique_labels:
        indices = np.where(labels == label)[0]
        selected_indices = np.random.choice(indices, n_samples, replace=True)
        downsampled_images.extend(images[selected_indices])
        downsampled_labels.extend(labels[selected_indices])

        return np.array(downsampled_images), np.array(downsampled_labels)

In [ ]:
images, labels = downsample(images, labels, 100)

In [ ]:
count_classes = np.unique(labels, return_counts=True)
plt.bar(count_classes[0], count_classes[1])

In [ ]:
display_sample(images=images, labels=labels)

### Data Augmentation

In [ ]:
augmentator = ImageAugmentation(
    rotation_range=40,
    height_shift_range=0.1,
    width_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.1,
)

images, lables = augmentator.flow(images, labels, num_augmentation=6)

In [ ]:
images.shape, labels.shape

In [ ]:
plt.figure(figsize=(20, 10))

for i in range(7):
    plt.subplot(1, 7, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap='gray')
    plt.xlabel('Original Image' if i == 0 else f"Augmented Image {i}")
plt.imshow()

### Preprocessing

In [ ]:
def skin_masking(images):
    skin_images = []
    for image in images:
        R_Frame = image[:, :, 0]
        G_Frame = image[:, :, 1]
        B_Frame = image[:, :, 2]
        BRG_Max = np.maximum.reduce([B_Frame, G_Frame, R_Frame])
        BRG_Min = np.minimum.reduce([B_Frame, G_Frame, R_Frame])
        Rule_1 = np.logical_and.reduce([R_Frame > 50, G_Frame > 40, B_Frame > 20, BRG_Max - BRG_Min > 13, abs(R_Frame - G_Frame) > 10, R_Frame > G_Frame, R_Frame > B_Frame])
        Rule_2 = np.logical_and.reduce([R_Frame > 220, G_Frame > 210, B_Frame > 170, abs(R_Frame - G_Frame) <= 15, R_Frame > B_Frame, G_Frame > B_Frame])
        mask = mask.astype(np.uint8) * 255
        skin = cv2.bitwise_and(image, image, mask = mask)
        skin_images.append(skin)
    return np.array(skin_images)

In [ ]:
skin_images = skin_masking(images)
display_sample(images=skin_images, labels=labels)

### Gray Scale

In [ ]:
gray_images = [cv2.cvtColor(image, cv2.COLOR_RGB2GRAY) for image in skin_images]
print(gray_images[0])

In [ ]:
display_sample(images=gray_images, labels=labels)

### Morphology

In [ ]:
morphology = Morphology(kernel=np.ones((3,3), np.uint8), num_workers=8)

In [ ]:
binary_mask = morphology.binary(images=gray_images, binary_threshold=1)
opening_mask = morphology.opening(images=binary_mask)

display_sample(images=opening_mask, labels=labels)

In [ ]:
def apply_mask(images, masks):
    masked_images = []
    for image, mask in zip(images, masks):
        mask = mask.astype(np.uint8) * 255
        masked_image = cv2.bitwise_and(image, image, mask=mask)
        masked_images.append(masked_image)
    return masked_images

In [ ]:
opening_images = apply_mask(gray_images, opening_mask)
display_sample(images=opening_images, labels = labels)

### Enhancement

In [ ]:
enchancer = Enhancement(num_workers=8)

In [ ]:
enhanced_images = enchancer.blur_images(blur_type='median', images=opening_images, kernel_size=3)
display_sample(images=enhanced_images, labels=labels)

### Find Contours

In [ ]:
def resize(image, size):
    height, width = image.shape[:2]
    new_height, new_width = size
    y_new, x_new = np.indices((new_height, new_width))
    y_new, x_new = (y_new * height // new_height).astype(int), (x_new * width // new_width).astype(int)
    image_resized = image[y_new, x_new]
    return image_resized

In [ ]:
def find_contours(images):
    results = []
    for image in images:
        binary_image = np.where(image > 87,  255, 0).astype(np.uint8)
        rows, cols = np.nonzero(binary_image)
        min_row, min_col = np.min(rows), np.min(cols)
        max_row, max_col = np.max(rows), np.max(cols)
        cropped_image = image[min_row:max_row+1, min_col:max_col+1]
        resized_image = resize(cropped_image, (75, 75))
        results.append(resized_image)
    return results

In [ ]:
contours_images = find_contours(enhanced_images)
display_sample(images=contours_images, labels=labels)

### Features Extraction

In [ ]:
dict_labels = {
    'finger_1' : 0,
    'finger_2' : 1,
    'finger_3' : 2,
    'finger_4' : 3,
    'finger_5' : 4,
}

In [ ]:
def extract_glcm_features(images, labels, distances=[5], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True):
    features = ['contrast', 'dissimilarity', 'homogenity', 'energy', 'correlation']
    angle_labels = ['0', '45', '90', '135']
    df_data = pd.DataFrame()
    for i, image in enumerate(images):
        image = image.astype(int)
        glcm = graycomatrix(image, distances, angles, levels, symmetric, normed)
        for feature in features:
            for angle, angle_label in zip(angles, angle_labels):
                feat_a = graycoprops(glcm, feature)[0, int(angle*4/np.pi)]
                df_data.loc[i, f'{feature}_{angle_label}'] = feat_a
        for angle, angle_label in zip(angles, angle_labels):
            asm = np.sum(glcm[:,:,0,int(angle*4/np.pi)]**2)
            entropy = -np.sum(glcm[:,:,0,int(angle*4/np.pi)]*np.log2(glcm[:,:,0,int(angle*4/np.pi)] + np.finfo(float).eps))
            df_data.loc[i, f'asm_{angle_label}'] = asm
            df_data.loc[i, f'entropy_{angle_label}'] = entropy
        df_data.loc[i, 'label'] = dict_labels[labels[i]]
    return df_data


In [ ]:
df_glcm = extract_glcm_features(contours_images, labels=labels)

In [ ]:
df_glcm

In [ ]:
df_glcm = pd.read_csv('glcm_features.csv')

In [ ]:
corr_glcm = df_glcm.corr()
plt.figure(figsize=(15,15))
sns.heatmap(corr_glcm, annot=True, cmap='coolwarm', cbar=False)
plt.tick_params(axis='both', which='major', labelsize=8)
plt.tick_params(axis='both', which='minor', labelsize=8)
plt.show()

In [ ]:
x, y = df_glcm.drop('label', axis=1), df_glcm['label']

In [ ]:
pca = PCA(n_components=25)
x_pca = pca.fit_transform(x)

### Splitting Data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_pca, y, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

### Normalization

In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

## Modeling

### KNN Classifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1, metric='cosine')
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print('Accuracy: ', accuracy_score(y_train, y_pred))

### SVM Classifier

In [ ]:
svm = SVC(kernel='rbf', C=30, gamma='auto')
svm.fit(x_train, y_train)
y_pred = svm.predict(x_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))

### Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=500, class_weight='balanced', n_jobs=1, max_depth=50, min_samples_leaf=1, min_samples_split=2, bootstrap=False, criterion='gini')
rfc.fit(x_train, y_train)
y_pred = rfc.predict(x_test)
print('Accuracy: ', accuracy_score(y_test, y_pred))

## Evaluation

In [ ]:
model_dict = {
    'KNN' : knn,
    'SVM' : svm,
    'RF' : rfc
}

df_eval = pd.DataFrame(columns=['Accuracy', 'Precision', 'Recall', 'F1 Score'], index=model_dict.keys())

In [ ]:
for model_name, model in model_dict.items():
    y_pred = model.predict(x_test)
    df_eval.loc[model_name, 'Accuracy'] = accuracy_score(y_test, y_pred)
    df_eval.loc[model_name, 'Precision'] = precision_score(y_test, y_pred, average='weighted')
    df_eval.loc[model_name, 'Recall'] = recall_score(y_test, y_pred, average='weighted')
    df_eval.loc[model_name, 'F1 Score'] = f1_score(y_test, y_pred, average='weighted')

In [ ]:
df_eval

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20, 5))
for i, model_name in enumerate(model_dict.keys()):
    y_pred = model_dict[model_name].predict(x_test)
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, cmap='coolwarm', cbar=False, ax=ax[i])
    ax[i].set_xlabel('Predicted')
    ax[i].set_ylabel('True')
    ax[i].set_title(model_name)
plt.show()